# Distributed analysis example: exhaustive ddG PSSM

## Analysis of delta score

In [1]:
import pandas
import matplotlib
import seaborn

/Users/weitzner/anaconda3/envs/pyrosettascripts_demo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/weitzner/anaconda3/envs/pyrosettascripts_demo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
result_frame = pandas.read_pickle('result_frame.pkl')

In [ ]:
matplotlib.rcParams['figure.figsize'] = [24.0, 8.0]

In [ ]:
seaborn.heatmap(
    result_frame.pivot("mutation_aa", "mutation_index", "delta_total_score"),
    cmap="RdBu_r", center=0, vmax=50)